# Fine Tuning modernBERT



---
## Install Dependencies

In [ ]:
import torch
from sentence_transformers.training_args import BatchSamplers
import pandas as pd
from datasets import load_dataset, concatenate_datasets
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

torch.set_float32_matmul_precision('high')

Mounted at /content/drive


**Login to Hugging Face**

Used for pushing model to the Hugging Face Hub and downloading gated models or datasets

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token="", add_to_git_credential=True)
#login(token=userdata.get('HF_TOKEN'), add_to_git_credential=True)


-----

------

---
## IMPORT DATA: Preparation


In [ ]:
# Load dataset from the Drive
#PUT THE ACTUAL PATH.
path = '/content/drive/Shareddrives/Master_Thesis/Data/final_dataset.csv'
dataset = pd.read_csv(path)

#split data
df_unseen_test = pd.read_csv('/content/drive/Shareddrives/Master_Thesis/Data/unseen_test.csv')
df_train = pd.read_csv('/content/drive/Shareddrives/Master_Thesis/Data/train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/Master_Thesis/Data/test.csv')
print(len(dataset))
print(len(df_unseen_test))
print(len(df_test))
print(len(df_train))
t=len(df_unseen_test)+len(df_test)+len(df_train)
print(t )

88775
210
8668
79897
88775


As we can see there we dont have the (anchor, positive, negative) triplets structure in the dataset. That is why from the **hard_negative_mining** notebook we used BM25 and the cosine similarity to obtain from each content, a negative question.

We just need to have the triplets structure in the Trian

In [ ]:
df_train

,id,paper id,title,categories,type,content,question
0,130,2501.00784,cloitre's self-generating sequence,"['math.co', 'cs.dm', 'cs.fl', 'math.nt']",theorems,Let $g_n$ be the number of $1$'s in the sequen...,What is the limit of the proportion of 1's in ...
1,265,2501.00809,initial ideals of weighted forms and the genus...,"['math.ac', 'math.ag']",theorems,\label{ThmConjAreTrue}\nConjectures \ref{Conj1...,Does the statement of \textbf{ThmConjAreTrue} ...
2,266,2501.00809,initial ideals of weighted forms and the genus...,"['math.ac', 'math.ag']",propositions,}\n\newcommand{\ep}{,\\emph{Is the statement \emph{If $X$ is a comp...
3,267,2501.00809,initial ideals of weighted forms and the genus...,"['math.ac', 'math.ag']",definitions,}\n\newcommand{\ed}{,Is the statement $\ed{True}$?
4,313,2501.00845,spectral spaces of normal subgroups,"['math.gr', 'math.gn']",theorems,\label{mth}\nLet $G$ be a group having a maxim...,Does the set $\mathcal{N}^+(G)$ of proper norm...
...,...,...,...,...,...,...,...
79892,40568,2501.15565,homogeneity of rearrangement-invariant norms,['math.fa'],definitions,\label{D:Delta}\n\tLet $\mathcal Q$ be an inde...,What is the relationship between the set $\Del...
79893,57464,2502.02511,$\mathcal{h}^{1}$ and $\mathrm{bmo}$ regularit...,"['math.ap', 'math.ca']",lemmas,"\label{lem:pseudosmooth}\nLet $a\in S^{m}_{1,1...","Is the operator $a(x,D)$ in the theorem \label..."
79894,20658,2501.07874,"constrained differential operators, sobolev in...","['math.fa', 'math.ap']",theorems,"\label{K-CZ}\n Let $\alpha \in (0,n)$ and supp...",What is the kernel estimate for the convolutio...
79895,47493,2501.18053,varieties of prime tropical ideals and the dim...,"['math.ag', 'math.ac']",propositions,\label{prop: Iprime-bendPrime}\r\nLet $I$ be a...,Is Bend(I) prime for any prime ideal I of the ...


In [ ]:
df_test

,id,paper id,title,categories,type,content,question
0,28,2501.00724,category o for quantum loop algebras,"['math.rt', 'math.qa']",definitions,\label{def:q-character}\n\n(\cite{FR}) The $q$...,What is the $q$-character of the representatio...
1,55,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],lemmas,\label{lm:bound_of_kappaBinvA}\n\tFor any $t >...,What is the probability that the condition $\k...
2,60,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],lemmas,[Lemma 3.6 of \citet{xiong2024accessible}]\lab...,What is the expression for $Q_{\bar{\Theta}}^{...
3,66,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],corollaries,\label{cor:complexity-random}\n\tLet $C_3 > 1$...,Does the probabilistic model with rPDHG achiev...
4,74,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],definitions,[$\eps$-optimal solutions]\n\tThe pair of prim...,What does it mean for a pair of primal-dual so...
...,...,...,...,...,...,...,...
8663,88749,2502.16067,the shifted convolution problem in function fi...,['math.nt'],theorems,\label{th4}\n Suppose that $h$ is a monic po...,What is the error term $E$ in the given formul...
8664,88754,2502.16067,the shifted convolution problem in function fi...,['math.nt'],lemmas,\label{residue_d}\nUnder the same assumptions ...,What is the formula for the main term of the V...
8665,88757,2502.16067,the shifted convolution problem in function fi...,['math.nt'],propositions,\label{prop:D2_func} Let $g \in \cM_{\geqs 1}...,What is the relationship between the function ...
8666,88760,2502.16067,the shifted convolution problem in function fi...,['math.nt'],propositions,\label{prop:Rvoronoi_notcoprime}\nLet $g \in \...,What is the relationship between the sum of th...


In [ ]:
import pickle
# step 1 import the dictionary that matches each question with the "negative" content
with open('/content/drive/Shareddrives/Master_Thesis/Fine-Tuning/Embeddings/hard_negatives_modernbert_base.pkl','rb') as f:
    hard_negs = pickle.load(f)

#  match from each
for idx, row in df_train.iterrows():
    source_id = row['id']
    matches = hard_negs.get(source_id, [])

    if matches:  # Only proceed if there is at least one match
        match_id = matches[0]
        question_row = df_train.loc[df_train['id'] == match_id, 'content']

        if not question_row.empty:


          df_train.at[idx, 'negative'] = question_row.values[0]

        else:
          df_train.at[idx, 'negative'] = None  # or some placeholder
    else:
        df_train.at[idx, 'negative'] = None  # or some placeholder



Clean the NANs

In [ ]:
print(len(df_train[df_train['negative'].isna()]))
len(df_train[df_train['negative'].isna()])
dataset_train=df_train[df_train['negative'].notna()]

21


In [ ]:
dataset_train

,id,paper id,title,categories,type,content,question,negative
0,130,2501.00784,cloitre's self-generating sequence,"['math.co', 'cs.dm', 'cs.fl', 'math.nt']",theorems,Let $g_n$ be the number of $1$'s in the sequen...,What is the limit of the proportion of 1's in ...,[] The tree $ \mathcal{T}_{ \infty}$ is the li...
1,265,2501.00809,initial ideals of weighted forms and the genus...,"['math.ac', 'math.ag']",theorems,\label{ThmConjAreTrue}\nConjectures \ref{Conj1...,Does the statement of \textbf{ThmConjAreTrue} ...,[{\cite[Corollary 2.2.2 with $p=3$]{BSY}}]\n ...
2,266,2501.00809,initial ideals of weighted forms and the genus...,"['math.ac', 'math.ag']",propositions,}\n\newcommand{\ep}{,\\emph{Is the statement \emph{If $X$ is a comp...,\label{prop:coherence}\n\tIf $X$ is a qcqs sch...
3,267,2501.00809,initial ideals of weighted forms and the genus...,"['math.ac', 'math.ag']",definitions,}\n\newcommand{\ed}{,Is the statement $\ed{True}$?,\label{homogeneous linear order is possible}\n...
4,313,2501.00845,spectral spaces of normal subgroups,"['math.gr', 'math.gn']",theorems,\label{mth}\nLet $G$ be a group having a maxim...,Does the set $\mathcal{N}^+(G)$ of proper norm...,If every maximal subgroup of a finite group $G...
...,...,...,...,...,...,...,...,...
79892,40568,2501.15565,homogeneity of rearrangement-invariant norms,['math.fa'],definitions,\label{D:Delta}\n\tLet $\mathcal Q$ be an inde...,What is the relationship between the set $\Del...,"What is the function $f(A_\lambda,\lambda)$ so..."
79893,57464,2502.02511,$\mathcal{h}^{1}$ and $\mathrm{bmo}$ regularit...,"['math.ap', 'math.ca']",lemmas,"\label{lem:pseudosmooth}\nLet $a\in S^{m}_{1,1...","Is the operator $a(x,D)$ in the theorem \label...",\label{hmexistssmall}\nLet $L$ be as in \eqref...
79894,20658,2501.07874,"constrained differential operators, sobolev in...","['math.fa', 'math.ap']",theorems,"\label{K-CZ}\n Let $\alpha \in (0,n)$ and supp...",What is the kernel estimate for the convolutio...,\label{Fredholm-powerCompact}\nLet $T$ be a bo...
79895,47493,2501.18053,varieties of prime tropical ideals and the dim...,"['math.ag', 'math.ac']",propositions,\label{prop: Iprime-bendPrime}\r\nLet $I$ be a...,Is Bend(I) prime for any prime ideal I of the ...,\label{Check}\nLet $R$ be a quasi-Gorenstein e...


WE NEED TO RENAME THE columns.
We then need to format the dataset into a structure expected in the upcoming training: `[anchor, positive, id]`. We remove the extraneous columns, rename our `question` and `text` columns, and add in a simple `id` column.

In [ ]:
#TRAIN
from datasets import Dataset
dataset_train = Dataset.from_pandas(dataset_train, preserve_index=False)
# Clean & Format Columns
dataset_train = dataset_train.rename_column("question", "anchor")
dataset_train = dataset_train.rename_column("content", "positive")
dataset_train = dataset_train.remove_columns([ "title", "type"]) # keep category , paper id
# Add an id column to the dataset
#dataset_train = dataset_train.add_column("id", range(len(dataset_train)))

#TEST:
df_test = Dataset.from_pandas(df_test, preserve_index=False)
df_test = df_test.rename_column("question", "anchor")
df_test = df_test.rename_column("content", "positive")
df_test = df_test.remove_columns([ "title", "type"])

In [ ]:
df_test

,id,paper id,title,categories,type,content,question
0,28,2501.00724,category o for quantum loop algebras,"['math.rt', 'math.qa']",definitions,\label{def:q-character}\n\n(\cite{FR}) The $q$...,What is the $q$-character of the representatio...
1,55,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],lemmas,\label{lm:bound_of_kappaBinvA}\n\tFor any $t >...,What is the probability that the condition $\k...
2,60,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],lemmas,[Lemma 3.6 of \citet{xiong2024accessible}]\lab...,What is the expression for $Q_{\bar{\Theta}}^{...
3,66,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],corollaries,\label{cor:complexity-random}\n\tLet $C_3 > 1$...,Does the probabilistic model with rPDHG achiev...
4,74,2501.00728,high-probability polynomial-time complexity of...,['math.oc'],definitions,[$\eps$-optimal solutions]\n\tThe pair of prim...,What does it mean for a pair of primal-dual so...
...,...,...,...,...,...,...,...
8663,88749,2502.16067,the shifted convolution problem in function fi...,['math.nt'],theorems,\label{th4}\n Suppose that $h$ is a monic po...,What is the error term $E$ in the given formul...
8664,88754,2502.16067,the shifted convolution problem in function fi...,['math.nt'],lemmas,\label{residue_d}\nUnder the same assumptions ...,What is the formula for the main term of the V...
8665,88757,2502.16067,the shifted convolution problem in function fi...,['math.nt'],propositions,\label{prop:D2_func} Let $g \in \cM_{\geqs 1}...,What is the relationship between the function ...
8666,88760,2502.16067,the shifted convolution problem in function fi...,['math.nt'],propositions,\label{prop:Rvoronoi_notcoprime}\nLet $g \in \...,What is the relationship between the sum of th...


In [ ]:
print(dataset_train)
print(df_test)

Dataset({
    features: ['id', 'paper id', 'categories', 'positive', 'anchor', 'negative'],
    num_rows: 79876
})
Dataset({
    features: ['id', 'paper id', 'categories', 'positive', 'anchor'],
    num_rows: 8668
})


In [ ]:
train_df = dataset_train.to_pandas()
test_df  = df_test.to_pandas()


train_df.to_json("train_dataset.json",
                 orient="records", lines=True, index=False)
test_df.to_json("test_dataset.json",
                orient="records", lines=True, index=False)

---
## Base Model Evaluation


In [ ]:
from sentence_transformers import SentenceTransformer, models

repo_id = "answerdotai/ModernBERT-base"   # tu checkpoint DAPT

#  Cargamos el encoder con los pesos DAPT
word_model = models.Transformer(repo_id )             # usa AutoModel + AutoTokenizer
#  Añadimos mean pooling
pooling = models.Pooling(
    word_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
)

#  Construimos un modelo SentenceTransformer
model = SentenceTransformer(modules=[word_model, pooling])


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

To run our base evaluations, we need to prepare the data slightly differently for the [InformationRetrievalEvaluator](https://sbert.net/docs/package_reference/sentence_transformer/evaluation.html#sentence_transformers.evaluation.InformationRetrievalEvaluator). This evaluator requires three key data structures:

1. A corpus dictionary mapping IDs to documents (`{paper id: theorem}`)
2. A queries dictionary mapping IDs to questions (`{query_id: query}`)
3. A relevant_docs dictionary specifying which corpus documents are relevant for each query (`{query_id: [paper id]}`)

To build these structures:
- We combine train and test datasets into a single corpus_dataset to ensure all text chunks are available during evaluation
- The corpus dictionary is created from the combined corpus_dataset, containing all the papers
- The queries dictionary is created only from the test_dataset, as we want to evaluate on unseen questions
- For the relevance mapping, we use paper id as the connecting key to identify which corpus documents contain the  **id** (theorem_id) relevant to each test query

In [ ]:
# Load train and test datasets from their respective JSON files
# These contain pairs of questions (anchors) and text chunks (positives)
train_df = pd.read_json("train_dataset.json", orient="records", lines=True)
test_df  = pd.read_json("test_dataset.json",  orient="records", lines=True)


# Combine train and test datasets into a single corpus (without the unseen data)
# This ensures we have all possible text chunks available for retrieval evaluation
# 2. Convert them to Hugging-Face Datasets
#    preserve_index=False keeps the column set identical to the CSV
train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset  = Dataset.from_pandas(test_df,  preserve_index=False)

# 3. Build the combined corpus

corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

# Convert datasets into dictionary format required by the InformationRetrievalEvaluator
# corpus: maps corpus IDs to their text chunks (documents)
corpus = dict(zip(corpus_dataset["id"], corpus_dataset["positive"]))
queries = dict(zip(test_dataset["id"],     test_dataset["anchor"]))


# Create a mapping between queries and their relevant documents
# This tells the evaluator which documents are correct matches for each query

#IMPORATN: WE CONSIDER THAT all content chunks (theorems, lemmas, etc) from the same paper can be considered relevant
DOC_ID_COL = "paper id"

# relevant_docs = {}
# for q_id, doc_key in zip(test_dataset["id"], test_dataset[DOC_ID_COL]):
#     # find all corpus rows that share the same doc_key
#     matching_ids = [
#         cid for cid, key in zip(corpus_dataset["id"], corpus_dataset[DOC_ID_COL])
#         if key == doc_key
#     ]
#     relevant_docs.setdefault(q_id, []).extend(matching_ids)




from collections import defaultdict

# Step 1: Build index of paper_id → list of content IDs
paper_to_ids = defaultdict(list)
for row in corpus_dataset:  # not .iterrows()
    paper_to_ids[row["paper id"]].append(row["id"])

# Step 2: For each test query, retrieve all content IDs for its paper
relevant_docs = {
    row["id"]: paper_to_ids[row["paper id"]]
    for row in test_dataset  # not .iterrows()
}



/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim
# Dimensions of interest

evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    precision_recall_at_k=[3, 5,30],
    ndcg_at_k=[10],
    mrr_at_k=[10],
    map_at_k=[100],
    accuracy_at_k=[1, 3, 5],
    relevant_docs=relevant_docs,
    name="TESTING",
    score_functions={"cosine": cos_sim},
)

In [ ]:
# Evaluate the model
base_results = evaluator(model)


W0531 07:52:29.108000 369 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


In [ ]:
base_results

{'TESTING_cosine_accuracy@1': 0.17766497461928935,
 'TESTING_cosine_accuracy@3': 0.21469773880941392,
 'TESTING_cosine_accuracy@5': 0.2350023073373327,
 'TESTING_cosine_precision@3': 0.10636825103830179,
 'TESTING_cosine_precision@5': 0.07932625749884632,
 'TESTING_cosine_precision@30': 0.024388555606829714,
 'TESTING_cosine_recall@3': 0.013011218217132213,
 'TESTING_cosine_recall@5': 0.015693734301277275,
 'TESTING_cosine_recall@30': 0.025973210058210277,
 'TESTING_cosine_ndcg@10': 0.07331679789165815,
 'TESTING_cosine_mrr@10': 0.20164983775389841,
 'TESTING_cosine_map@100': 0.01742706094887197}

{'TESTING_cosine_accuracy@1': 0.17766497461928935,
 'TESTING_cosine_accuracy@3': 0.21469773880941392,
 'TESTING_cosine_accuracy@5': 0.2350023073373327,
 'TESTING_cosine_accuracy@10': 0.2596908167974158,
 'TESTING_cosine_precision@1': 0.17766497461928935,
 'TESTING_cosine_precision@3': 0.10636825103830179,
 'TESTING_cosine_precision@5': 0.07932625749884632,
 'TESTING_cosine_precision@10': 0.05111905860636825,
 'TESTING_cosine_recall@1': 0.007872651104495367,
 'TESTING_cosine_recall@3': 0.013011218217132213,
 'TESTING_cosine_recall@5': 0.015693734301277275,
 'TESTING_cosine_recall@10': 0.019188661012940474,
 'TESTING_cosine_ndcg@10': 0.07331679789165815,
 'TESTING_cosine_mrr@10': 0.20164983775389841,
 'TESTING_cosine_map@100': 0.01742706094887197}

---
## Training



### BASE MODERNBERT

In [ ]:
# load model with SDPA for using Flash Attention 2
from sentence_transformers import SentenceTransformer, models, SentenceTransformerModelCardData
import torch
from sentence_transformers.models import Normalize, Pooling, Transformer
# Por si tienes hardware compatible con Flash Attention 2
torch.set_float32_matmul_precision("high")

repo_id = "answerdotai/ModernBERT-base"

#  Cargamos el encoder con los pesos DAPT
word_model = models.Transformer(repo_id )
#  Añadimos mean pooling
pooling = models.Pooling(
    word_model.get_word_embedding_dimension(),
     pooling_mode_mean_tokens=True,
)

normalize = Normalize()
#  Construimos un modelo SentenceTransformer
model = SentenceTransformer(modules=[word_model, pooling, normalize ],model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="ModernBERT base",
    ))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
model

Next is defining our loss function. Loss functions are what's used to guide the model towards improvements at train time, generally comparing current performance with expected performance, calculating the difference, and then the value determines the direction we optimize towards.

Sentence Transformers offers [many different loss functions](https://sbert.net/docs/sentence_transformer/loss_overview.html) for various scenarios. Given our commitment to MRL training, we will need not only a base loss function, but an additional adapter.

TripletLoss

We wrap this with [`MatryoshkaLoss`](https://sbert.net/docs/package_reference/sentence_transformer/losses.html#matryoshkaloss) to enable multi-dimensional embedding training, allowing for dynamic dimensionality reduction at inference time without requiring retraining.

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerModelCardData, SentenceTransformerTrainingArguments, SentenceTransformerTrainer
#from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
#from sentence_transformers.util import cos_sim
from sentence_transformers.losses import MultipleNegativesRankingLoss, TripletLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.losses import TripletDistanceMetric

# Initial Loss
train_loss = MultipleNegativesRankingLoss(model)
#train_loss = TripletLoss(model,distance_metric=TripletDistanceMetric.COSINE, triplet_margin = 0.1)


Below are the defined training hyperparameters. These are taken directly from the aforementioned [Philipp Schmid's original blogpost](https://www.philschmid.de/fine-tune-embedding-model-for-rag#4-fine-tune-embedding-model-with-sentencetransformerstrainer). It is worth testing various combinations of hyperparameters for optimal performance, but for the sake of this demonstration we will default to Philipp's provided arguments.

In [ ]:
# Training Arguments
args = SentenceTransformerTrainingArguments(
    output_dir="/content/drive/Shareddrives/Master_Thesis/Models/FT_base_negatives", # output directory and hugging face model ID
    num_train_epochs=4,                                        # number of epochs
    per_device_train_batch_size=16,                            # train batch size
    gradient_accumulation_steps=8,                             # for a global batch size of 512
    per_device_eval_batch_size=16,                             # evaluation batch size
    warmup_ratio=0.1,                                          # warmup ratio
    learning_rate=2e-5,                                        # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                                # use cosine learning rate scheduler
    optim="adamw_torch_fused",                                 # use fused adamw optimizer
    tf32=True,                                                 # use tf32 precision
    bf16=True,                                                 # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,                 # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                                     # evaluate after each epoch
    save_strategy="epoch",                                     # save after each epoch
    logging_steps=10,                                          # log every 10 steps
    save_total_limit=3,                                        # save only the last 3 models
    load_best_model_at_end=True,                               # load the best model when training ends
    metric_for_best_model="eval_TESTING_cosine_ndcg@10",       # Optimizing for the best ndcg@10 score for the 128 dimension
    report_to="none"                                           # Turning off training logging for now, input 'wandb' etc. if desired.
)

Finally, package our model, training arguments, dataset, loss function and evaluator together into a `SentenceTransformerTrainer`

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset.select_columns(
        [ "anchor","positive", "negative"]
    ),
    loss=train_loss,
    evaluator=evaluator,
)

Start the training run!

In [ ]:
import math

total_examples = len(train_dataset)
global_batch_size = args.per_device_train_batch_size * args.gradient_accumulation_steps
steps_per_epoch = math.ceil(total_examples / global_batch_size)
total_steps = steps_per_epoch * args.num_train_epochs

print(f"Estimated steps per epoch: {steps_per_epoch}")
print(f"Estimated total steps: {total_steps}")


In [ ]:
import torch
import os

# Set memory management behavior
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Clean up any old memory references
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Now proceed to import and initialize models, datasets, etc.
from sentence_transformers import SentenceTransformer, models


In [ ]:
# Start training
trainer.train()

# Save the best model based on our criteria
trainer.save_model('/content/drive/Shareddrives/Master_Thesis/Models/FT_base_negatives')

In [ ]:
evaluator(model)

In [ ]:
output_dir='/content/drive/Shareddrives/Master_Thesis/Models/FT_base'
trainer.save_model(output_dir)

In [ ]:
# Upload model to hub
trainer.model.push_to_hub("Master-thesis-NAP/ModernBERT-base-finetuned-hard_negatives")

---
## Evaluating Trained Model

In [ ]:
from sentence_transformers import SentenceTransformer, models

repo_id = "Master-thesis-NAP/ModernBERT-base-finetuned"   # tu checkpoint DAPT

#  Cargamos el encoder con los pesos DAPT
word_model = models.Transformer(repo_id )             # usa AutoModel + AutoTokenizer
#  Añadimos mean pooling
pooling = models.Pooling(
    word_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
)

#  Construimos un modelo SentenceTransformer
model = SentenceTransformer(modules=[word_model, pooling])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
ft_results=evaluator(model)

In [ ]:
ft_results

{'TESTING_cosine_accuracy@1': 0.8654822335025381,
 'TESTING_cosine_accuracy@3': 0.9149746192893401,
 'TESTING_cosine_accuracy@5': 0.9282418089524689,
 'TESTING_cosine_precision@3': 0.6053299492385786,
 'TESTING_cosine_precision@5': 0.48627134287032764,
 'TESTING_cosine_precision@30': 0.17020458391016768,
 'TESTING_cosine_recall@3': 0.08219095045114123,
 'TESTING_cosine_recall@5': 0.10574905214625942,
 'TESTING_cosine_recall@30': 0.19040101210624102,
 'TESTING_cosine_ndcg@10': 0.4428364819184664,
 'TESTING_cosine_mrr@10': 0.8931939042345122,
 'TESTING_cosine_map@100': 0.15987022599453457}

In [ ]:
df_unseen = Dataset.from_pandas(df_unseen_test, preserve_index=False)
df_unseen = df_unseen.rename_column("question", "anchor")
df_unseen = df_unseen.rename_column("content", "positive")
df_unseen = df_unseen.remove_columns([ "title", "type"])
# unseen_df = df_unseen.to_pandas()
# unseen_df.to_json("unseen_test_dataset.json", orient="records", lines=True, index=False)

corpus_dataset_unseen = concatenate_datasets([train_dataset, test_dataset, df_unseen])
corpus_unseen = dict(zip(corpus_dataset_unseen["id"], corpus_dataset_unseen["positive"]))
queries_unseen = dict(zip(df_unseen["id"],     df_unseen["anchor"]))

DOC_ID_COL = "paper id"

from collections import defaultdict

# Step 1: Build index of paper_id → list of content IDs
paper_to_ids = defaultdict(list)
for row in corpus_dataset_unseen:  # not .iterrows()
    paper_to_ids[row["paper id"]].append(row["id"])

# Step 2: For each test query, retrieve all content IDs for its paper
relevant_docs_unseen = {
    row["id"]: paper_to_ids[row["paper id"]]
    for row in df_unseen
}
evaluator_unseen = InformationRetrievalEvaluator(
    queries=queries_unseen,
    corpus=corpus_unseen,
    precision_recall_at_k=[3, 5,30],
    ndcg_at_k=[10],
    mrr_at_k=[10],
    map_at_k=[100],
    accuracy_at_k=[1, 3, 5],
    relevant_docs=relevant_docs_unseen,
    name="TESTING",
    score_functions={"cosine": cos_sim},
)

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
unseen_results= evaluator_unseen(model)

In [ ]:
unseen_results

{'TESTING_cosine_accuracy@1': 0.819047619047619,
 'TESTING_cosine_accuracy@3': 0.861904761904762,
 'TESTING_cosine_accuracy@5': 0.8857142857142857,
 'TESTING_cosine_precision@3': 0.6063492063492063,
 'TESTING_cosine_precision@5': 0.4923809523809524,
 'TESTING_cosine_precision@30': 0.14888888888888893,
 'TESTING_cosine_recall@3': 0.1299319727891156,
 'TESTING_cosine_recall@5': 0.17585034013605438,
 'TESTING_cosine_recall@30': 0.319047619047619,
 'TESTING_cosine_ndcg@10': 0.43164437775888315,
 'TESTING_cosine_mrr@10': 0.8483125472411187,
 'TESTING_cosine_map@100': 0.2627941269534501}